# Stitching Tutorial

The Workflow of the Stitching Pipeline can be seen in the following. Note that the image comes from the [OpenCV Documentation](https://docs.opencv.org/3.4/d1/d46/group__stitching.html).

With the following block, we allow displaying resulting images within the notebook:

In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

With the following block, we load the correct img paths to the used image sets:

In [2]:
from pathlib import Path
def get_image_paths(img_set):
    return [str(path.relative_to('.')) for path in Path('../../data/sequential_training').rglob(f'{img_set}*')]

sequential_imgs = get_image_paths('')
sequential_imgs = sorted(sequential_imgs)[32:40]

In [3]:
len(sequential_imgs)

8

## Resize Images

The first step is to resize the images to medium (and later to low) resolution. The class which can be used is the `ImageHandler` class. If the images should not be stitched on full resolution, this can be achieved by setting the `final_megapix` parameter to a number above 0. 

`ImageHandler(medium_megapix=0.6, low_megapix=0.1, final_megapix=-1)`

In [4]:
from stitching.image_handler import ImageHandler

img_handler = ImageHandler()
img_handler.set_img_names(sequential_imgs)

medium_imgs = list(img_handler.resize_to_medium_resolution())
final_imgs = list(img_handler.resize_to_final_resolution())

**NOTE:** Everytime `list()` is called in this notebook means that the function returns a generator (generators improve the overall stitching performance). To get all elements at once we use `list(generator_object)`  

In [5]:
final_size = img_handler.get_image_size(final_imgs[0])

## Find Features

On the medium images, we now want to find features that can describe conspicuous elements within the images which might be found in other images as well. The class which can be used is the `FeatureDetector` class.

`FeatureDetector(detector='orb', nfeatures=500)`

In [6]:
from stitching.feature_detector import FeatureDetector

finder = FeatureDetector()
features = [finder.detect_features(img) for img in medium_imgs]

## Match Features

Now we can match the features of the pairwise images. The class which can be used is the FeatureMatcher class.

`FeatureMatcher(matcher_type='homography', range_width=-1)`

In [7]:
from stitching.feature_matcher import FeatureMatcher

matcher = FeatureMatcher()
matches = matcher.match_features(features)

It can be seen that:

- image 1 has a high matching confidence with image 2 and low confidences with image 3 and 4
- image 2 has a high matching confidence with image 1 and image 3 and low confidences with image 4
- image 3 has a high matching confidence with image 2 and low confidences with image 1 and 4
- image 4 has low matching confidences with image 1, 2 and 3

With a `confidence_threshold`, which is introduced in detail in the next step, we can plot the relevant matches with the inliers:

## Camera Estimation, Adjustion and Correction

With the features and matches we now want to calibrate cameras which can be used to warp the images so they can be composed correctly. The classes which can be used are `CameraEstimator`, `CameraAdjuster` and `WaveCorrector`:

```
CameraEstimator(estimator='homography')
CameraAdjuster(adjuster='ray', refinement_mask='xxxxx')
WaveCorrector(wave_correct_kind='horiz')
```

In [8]:
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector

camera_estimator = CameraEstimator()
camera_adjuster = CameraAdjuster()
wave_corrector = WaveCorrector()

cameras = camera_estimator.estimate(features, matches)
cameras = camera_adjuster.adjust(features, matches, cameras)
cameras = wave_corrector.correct(cameras)

## Warp Images

With the obtained cameras we now want to warp the images itself into the final plane. The class which can be used is the `Warper` class:

`Warper(warper_type='spherical', scale=1)`

In [9]:
from stitching.warper import Warper

warper = Warper()

At first, we set the the medium focal length of the cameras as scale:

In [10]:
warper.set_scale(cameras)

Warp final resolution images

In [11]:
final_sizes = img_handler.get_final_img_sizes()
camera_aspect = img_handler.get_medium_to_final_ratio()    # since cameras were obtained on medium imgs

warped_final_imgs = list(warper.warp_images(final_imgs, cameras, camera_aspect))
warped_final_masks = list(warper.create_and_warp_masks(final_sizes, cameras, camera_aspect))
final_corners, final_sizes = warper.warp_rois(final_sizes, cameras, camera_aspect)

## Excursion: Timelapser

The Timelapser functionality is a nice way to grasp how the images are warped into a final plane. The class which can be used is the `Timelapser` class:

`Timelapser(timelapse='no')`

In [12]:
from stitching.timelapser import Timelapser

def get_first_nonzero_pixel(ndarr):
    return np.argwhere(ndarr > 0)[0][:2]

def get_center(corner, size):
    return (round(corner[0] + size[1] / 2), round(corner[1] + size[0] / 2))

timelapser = Timelapser('as_is')
timelapser.initialize(final_corners, final_sizes)
dex = 1
centers = []
for img, corner, size in zip(warped_final_imgs, final_corners, final_sizes):
    timelapser.process_frame(img, corner)
    frame = timelapser.get_frame()
    centers.append(get_center(get_first_nonzero_pixel(frame), size))

print(centers)
print(len(centers))

[(112, 224), (224, 112), (224, 224), (112, 336), (112, 448), (224, 336), (224, 448), (112, 560)]
8
